# AŞAMA 1 VERİNİN ÇEKİLMESİ

# Bu Bölümün İçeriği

## 1. Gerekli Kütüphanelerin Eklenmesi
## 2. Selenium İle Veri Çekme İşlemi
## 3. Verinin Düzenlenmesi


### Veri çekme işleminde her sayfa için url yi kendim aldım.

In [17]:
# her ilçeden 4 sayfa çek ve csv ye kaydet

### Bu bölümde gerekli olan kütüphaneleri ekliyoruz

In [ ]:
from selenium import webdriver
import time
import pandas as pd

### Veri çekme işlemi için gerekli olan kodlara başlıyoruz

In [5]:
# Bu hazır fonksyonumuz adından da anlaşılacağı üzere listeyi string türüne çeviriyor.
def listToString(s):  
    
    # initialize an empty string 
    str1 = " " 
    
    # return string   
    return (str1.join(s)) 

# Bu aşamada selenium ile veri çekmeye başlıyoruz.

# chrome driverin yolunu veriyoruz. Selenium u kullanmak için gereklidir.
driver_path = "D:\PROGRAM SETUP\chromedriver.exe"
# browser adında bi değişken oluşturuyoruz ve driver yolumuzu veriyoruz. Bunu ortam değişkenlerine eklememiz gerekiyor öncesinde
browser = webdriver.Chrome(driver_path)
# browser get ile verdiğimiz adres de ki web sitesine gitmiş oluyoruz.
browser.get("https://www.emlakjet.com/satilik-konut/istanbul-sariyer/3/")

# şimdi bu sayfa içinde her linke tıklayarak verileri çekeceğiz sonra o sayfadan çıkıp bi altında ki evin linke girip onun 
# verilerini alarak sırayla bu işlemi devam ettireceğiz.

# Şuan da web sitesinin arayüzündeyiz. Amacımız evlerin detaylarına bakmak için linke tıklamak. Bunu da xpathlere göre yapacağız
# neden hepsini tek döngü de yapmadık. çünkü arada reklamlar olduğu için pathler karışıyor. Reklamları atlamak için
# böyle bi yol izledim.
i = 1
while i<=2:
    # tıkla adında değişkene find diyerek bu xpath de 1. olana gidiyor.
    tıkla = browser.find_element_by_xpath("//*[@id='__next']/div[3]/div[1]/div/div[5]/div[1]/div[1]/div["+str(i)+"]")
    # click diyerek de bu linke tıklıyor ve evin detaylarına bakmak için sayfaya girmiş oluyoruz.
    tıkla.click()
    # özelliklerin hepsini almak için bu sefer bu kod ile orda ki css bloğuna göre veriyi çekeceğiz.
    elements = browser.find_elements_by_css_selector("._2xeaw")
    # özellikleri bir önceki kodda aldık şimdi de evin fiyatını alıyoruz ve değişkene kaydediyoruz.
    fiyatlar = browser.find_elements_by_css_selector("._34630")
    
    # 2 tane boş liste oluşturuyoruz. Elimizde ki dağınık veriyi düzenleyip listeye atacağız.
    detaylar = []
    fiyat = []
    # çektiğimiz veriler text string olmadığı için verileri stringe çevirip demin oluşturduğumuz boş listeye atıyoruz.
    # unutmayalım ki verileri çektiğiniz de liste olarak gelir.
    for i in fiyatlar:
        print(i.text)
        fiyat.append(i.text)
    # özellikleri de stringe çevirerek listeye atıyoruz.
    for i in elements:
        print(i.text)
        detaylar.append(i.text)
    # elimizde string olan liste halinde veriler var. Önceden oluşturduğumuz listeden stringe çevirme fonksyonuyla stringe 
    # çeviyoruz.
    det_str = listToString(detaylar)
    # string verimizi split ile satırlara göre ayırıyoruz. Ve ayrı adında değişkene atıyoruz.
    ayrı = det_str.split("\n")
    # demin oluşturduğumuz ayrı değişkenini DataFrame çeviriyoruz. 
    df = pd.DataFrame(ayrı)
    # Verimizde ki gereksiz verilerden kurtulmak için onları ayırıyoruz.
    df_yeni = df.iloc[7:53]
    df_yeni.iloc[45] = df_yeni.iloc[45].str[0:3]
    # değişiklikler yaptığımız için indexleri resetliyoruz.
    df_yeni = df_yeni.reset_index()
    # gereksiz oluşan şndex sütununu siliyoruz.
    df_yeni.drop("index", axis = 1, inplace = True)
    # verilerimizi tekrar listeye çeviriyorum.
    df_liste = df_yeni.values.tolist()
    # içerikler adında boş liste oluşturuyprum amacım şu şuan elimizde ki veri sütun adlarını ve içeriklerini barındırıyor.
    # içerikleri sütun adlarından ayırmak için bu işlemi yapıyoruz.
    içerikler =[]
    i = 1
    while i <= 45:
        print(df_liste[i])
        içerikler.append(df_liste[i])
        i = i+2
    # Hatırlarsanız fiyatı başka bi yerde çekmiştik ve bunu da içeriklere yeni bi sütuna ekliyoruz.
    # öncesinde fiyat değişkeninde ki gereksiz verileri siliyoruz. TL yazısını ve strip ile gereksiz boşlukları
    fiyat_sade = fiyat[1].strip()
    fiyat_sade = fiyat_sade.replace("TL","")
    # içerikler kısmına fiyatı ekliyoruz.
    içerikler.append([fiyat_sade])
    # şuan liste halinde olan verimizi Df e çeviriyoruz ve transpozunu alıyoruz.
    df_içerikler = pd.DataFrame(içerikler).T
    # verimizi csv dosyasına çeviriyoruz ve modu append yapıyoruz. Çünkü diğer çektiğimiz verileri de oraya ekleyeceğiz.
    # eğer mode = a yapmassak diğer gelen veriyi üzerine yazacaktır.
    df_içerikler.to_csv(r"zingat3.csv",encoding="utf-8",index=False, mode="a")
    
    # i değişkeni sayfada ki evlerin xpath de ki sayısıydı
    i = i+1
    # bu kod ile bi önceki sayfaya geçiyoruz.
    browser.execute_script("window.history.go(-1)")

    
print("ilk kısım çalıştı")

j = 4
while j <= 7:
    tıkla = browser.find_element_by_xpath("//*[@id='__next']/div[3]/div[1]/div/div[5]/div[1]/div[1]/div["+str(j)+"]")
    tıkla.click()
    
    elements = browser.find_elements_by_css_selector("._2xeaw")

    fiyatlar = browser.find_elements_by_css_selector("._34630")
    
    detaylar = []
    fiyat = []
    
    for i in fiyatlar:
        print(i.text)
        fiyat.append(i.text)
    
    for i in elements:
        print(i.text)
        detaylar.append(i.text)
    
    det_str = listToString(detaylar)
    ayrı = det_str.split("\n")
    
    df = pd.DataFrame(ayrı)
    df_yeni = df.iloc[7:53]
    df_yeni.iloc[45] = df_yeni.iloc[45].str[0:3]
    
    df_yeni = df_yeni.reset_index()
    df_yeni.drop("index", axis = 1, inplace = True)
    df_liste = df_yeni.values.tolist()
    
    içerikler =[]
    i = 1
    while i <= 45:
        print(df_liste[i])
        içerikler.append(df_liste[i])
        i = i+2
        
    fiyat_sade = fiyat[1].strip()
    fiyat_sade = fiyat_sade.replace("TL","")
    içerikler.append([fiyat_sade])
    df_içerikler = pd.DataFrame(içerikler).T
    
    df_içerikler.to_csv(r"zingat3.csv",encoding="utf-8",index=False, mode = "a")

    
    j = j+1
    browser.execute_script("window.history.go(-1)")

print("ikinci kısım çalıştı")


    
k = 9
while k<=10:
    tıkla = browser.find_element_by_xpath("//*[@id='__next']/div[3]/div[1]/div/div[5]/div[1]/div[1]/div["+str(k)+"]")
    tıkla.click()
    
    print("üçüncü ksıım 1")
    
    elements = browser.find_elements_by_css_selector("._2xeaw")

    fiyatlar = browser.find_elements_by_css_selector("._34630")
    
    detaylar = []
    fiyat = []
    
    for i in fiyatlar:
        print(i.text)
        fiyat.append(i.text)
    
    for i in elements:
        print(i.text)
        detaylar.append(i.text)
    
    det_str = listToString(detaylar)
    ayrı = det_str.split("\n")
    
    df = pd.DataFrame(ayrı)
    df_yeni = df.iloc[7:53]
    df_yeni.iloc[45] = df_yeni.iloc[45].str[0:3]
    
    df_yeni = df_yeni.reset_index()
    df_yeni.drop("index", axis = 1, inplace = True)
    df_liste = df_yeni.values.tolist()
    
    içerikler =[]
    i = 1
    while i <= 45:
        print(df_liste[i])
        içerikler.append(df_liste[i])
        i = i+2
        
    fiyat_sade = fiyat[1].strip()
    fiyat_sade = fiyat_sade.replace("TL","")
    içerikler.append([fiyat_sade])
    df_içerikler = pd.DataFrame(içerikler).T
    
    df_içerikler.to_csv(r"zingat3.csv",encoding="utf-8",index=False,mode="a")
    
    
    
    k = k+1
    browser.execute_script("window.history.go(-1)")
    
print("üçüncü kısım çalıştı")

l = 12
while l<=13:
    tıkla = browser.find_element_by_xpath("//*[@id='__next']/div[3]/div[1]/div/div[5]/div[1]/div[1]/div["+str(l)+"]")
    tıkla.click()
    
    elements = browser.find_elements_by_css_selector("._2xeaw")

    fiyatlar = browser.find_elements_by_css_selector("._34630")
    
    detaylar = []
    fiyat = []
    
    for i in fiyatlar:
        print(i.text)
        fiyat.append(i.text)
    
    for i in elements:
        print(i.text)
        detaylar.append(i.text)
    
    det_str = listToString(detaylar)
    ayrı = det_str.split("\n")
    
    df = pd.DataFrame(ayrı)
    df_yeni = df.iloc[7:53]
    df_yeni.iloc[45] = df_yeni.iloc[45].str[0:3]
    
    df_yeni = df_yeni.reset_index()
    df_yeni.drop("index", axis = 1, inplace = True)
    df_liste = df_yeni.values.tolist()
    
    içerikler =[]
    i = 1
    while i <= 45:
        print(df_liste[i])
        içerikler.append(df_liste[i])
        i = i+2
        
    fiyat_sade = fiyat[1].strip()
    fiyat_sade = fiyat_sade.replace("TL","")
    içerikler.append([fiyat_sade])
    df_içerikler = pd.DataFrame(içerikler).T
    
    df_içerikler.to_csv(r"zingat3.csv",encoding="utf-8",index=False,mode="a")
    
    
    
    l = l+1
    browser.execute_script("window.history.go(-1)")
    
print("dördüncü kısım çalıştı")
    
m = 15
while m<=30:
    tıkla = browser.find_element_by_xpath("//*[@id='__next']/div[3]/div[1]/div/div[5]/div[1]/div[1]/div["+str(m)+"]")
    tıkla.click()
    
    elements = browser.find_elements_by_css_selector("._2xeaw")

    fiyatlar = browser.find_elements_by_css_selector("._34630")
    
    detaylar = []
    fiyat = []
    
    for i in fiyatlar:
        print(i.text)
        fiyat.append(i.text)
    
    for i in elements:
        print(i.text)
        detaylar.append(i.text)
    
    det_str = listToString(detaylar)
    ayrı = det_str.split("\n")
    
    df = pd.DataFrame(ayrı)
    df_yeni = df.iloc[7:53]
    df_yeni.iloc[45] = df_yeni.iloc[45].str[0:3]
    
    df_yeni = df_yeni.reset_index()
    df_yeni.drop("index", axis = 1, inplace = True)
    df_liste = df_yeni.values.tolist()
    
    içerikler =[]
    i = 1
    while i <= 45:
        print(df_liste[i])
        içerikler.append(df_liste[i])
        i = i+2
        
    fiyat_sade = fiyat[1].strip()
    fiyat_sade = fiyat_sade.replace("TL","")
    içerikler.append([fiyat_sade])
    df_içerikler = pd.DataFrame(içerikler).T
    
    
    df_içerikler.to_csv(r"zingat3.csv",encoding="utf-8",index=False,mode="a")
    
    print("beşinci kısım 1")
    m = m+1
    browser.execute_script("window.history.go(-1)")
    
print("beşinci kısım son 1 ")




1.585.000TL
fullscreen
İlanı Haritada Görüntüle
İlan Numarası
8504856
İlan Oluşturma Tarihi
19 Temmuz 2020
İlan Güncelleme Tarihi
08 Ağustos 2020
Türü
Konut
Kategorisi
Satılık
Yapı Tipi
Betonarme
Net Metrekare
185 M2
Brüt Metrekare
220 M2
Salon Metrekare
Belirtilmemiş
Oda Sayısı
3+1
Binanın Yaşı
11-20
Bulunduğu Kat
Bahçe Dublex
Binanın Kat Sayısı
5
Isıtma Tipi
Merkezi Doğalgaz
Aidat
550 TL
Kira Getirisi
Belirtilmemiş
Banyo Sayısı
2
WC Sayısı
2
Site İçerisinde
Evet
Eşya Durumu
Boş
Kullanım Durumu
Mal Sahibi Oturuyor
Fiyat Durumu
Genel Fiyat
Yapı Durumu
İkinci El
Krediye Uygunluk
KREDIYE UYGUN
Yatırıma Uygunluk
Yatırıma Uygun
Takas
Yok
 220 M2 OLAN BAHÇE DUBLEKSİMİZ 4+1 DEN 3+1 E ÇEVRİLMİŞ YATAK ODASINA GİYİNME ODASI OLARAK EKLENMİŞTİR. 
KÖŞE BAHÇE OLMASI SEBEBİYLE FERAH VE KEYİFLİ BİR YAŞAM SUNMAKTADIR.
7/24 GÜVENLİK HİZMETİ SUNULMAKTADIR.
SARIYER TÜNELE2,5km
HACIOSMAN METRO İSTASYONUNA  7km
MASLAK 9km
KOÇ ÜNİVERSİTESİ 2km
HAVAALANI 45km
İETT 150,151,152,154 NOLU HATLAR SİTENİN ÖNÜNDEN

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


ilk kısım çalıştı

6.500.000TL
fullscreen
İlanı Haritada Görüntüle
İlan Numarası
8504333
İlan Oluşturma Tarihi
19 Temmuz 2020
İlan Güncelleme Tarihi
08 Ağustos 2020
Türü
Konut
Kategorisi
Satılık
Yapı Tipi
Betonarme
Net Metrekare
160 M2
Brüt Metrekare
185 M2
Salon Metrekare
Belirtilmemiş
Oda Sayısı
3+1
Binanın Yaşı
20 Ve Üzeri
Bulunduğu Kat
4. Kat
Binanın Kat Sayısı
6
Isıtma Tipi
Kombi Doğalgaz
Aidat
Belirtilmemiş
Kira Getirisi
Belirtilmemiş
Banyo Sayısı
1
WC Sayısı
Belirtilmemiş
Site İçerisinde
Hayır
Eşya Durumu
Belirtilmemiş
Kullanım Durumu
Mal Sahibi Oturuyor
Fiyat Durumu
Genel Fiyat
Yapı Durumu
İkinci El
Krediye Uygunluk
KREDIYE UYGUN
Yatırıma Uygunluk
Bilinmiyor
Takas
Yok
MERKEZİ YERLERE ULAŞIM SORUNU OLMAYAN
TOPLU TAŞIMA ARAÇLARINA YÜRÜME MESAFESİNDE
DENİZ MANZARALI ,SALON VE ODALARI GENİŞ VE GÜNEŞ GÖREN
BÖLGEDE FIRSAT FİATLI  3,5  ODALI DAİRE 
DETAYLI BİLGİ VE RANDEVU TEL
İç Özellikler
done
ADSL
done
Spot Işık
done
Duvar Kağıdı
done
Kartonpiyer
done
Gömme Dolap
done
Çelik Kapı
do

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



5.000.000TL
fullscreen
İlanı Haritada Görüntüle
İlan Numarası
8503649
İlan Oluşturma Tarihi
19 Temmuz 2020
İlan Güncelleme Tarihi
05 Ağustos 2020
Türü
Konut
Kategorisi
Satılık
Yapı Tipi
Betonarme
Net Metrekare
180 M2
Brüt Metrekare
260 M2
Salon Metrekare
30 M2
Oda Sayısı
6+2
Binanın Yaşı
20 Ve Üzeri
Bulunduğu Kat
Müstakil
Binanın Kat Sayısı
4
Isıtma Tipi
Kombi Doğalgaz
Aidat
Belirtilmemiş
Kira Getirisi
Belirtilmemiş
Banyo Sayısı
2
WC Sayısı
3
Site İçerisinde
Hayır
Eşya Durumu
Eşyalı
Kullanım Durumu
Mal Sahibi Oturuyor
Fiyat Durumu
Genel Fiyat
Yapı Durumu
İkinci El
Krediye Uygunluk
KREDIYE UYGUN
Yatırıma Uygunluk
Belirtilmemiş
Takas
Yok
  Yeniköy Köybaşı caddesine 25 metre mesafede yer alan mülk 2. Derece tarihi eser kayıtlı olup, 1990 yılında aslına uygun olarak renöve edilmiştir.
6+2 Oturum planında olan mülk, 2 mutfak, 2 banyo, 3 wc, 4 balkon, 1 ön veranda, 1 arka bahçe alanından oluşmaktadır. Özellikle üst 2 katından 2 açılı boğaz manzarası bulunmaktadır. 
Tarihi dokulu sokakları, 

...

üçüncü ksıım 1

1.025.000TL
fullscreen
İlanı Haritada Görüntüle
İlan Numarası
8498688
İlan Oluşturma Tarihi
18 Temmuz 2020
İlan Güncelleme Tarihi
08 Ağustos 2020
Türü
Konut
Kategorisi
Satılık
Yapı Tipi
Betonarme
Net Metrekare
135 M2
Brüt Metrekare
150 M2
Salon Metrekare
25 M2
Oda Sayısı
4+1
Binanın Yaşı
20 Ve Üzeri
Bulunduğu Kat
4. Kat
Binanın Kat Sayısı
5
Isıtma Tipi
Kombi Doğalgaz
Aidat
Belirtilmemiş
Kira Getirisi
3.000 TL
Banyo Sayısı
2
WC Sayısı
2
Site İçerisinde
Hayır
Eşya Durumu
Boş
Kullanım Durumu
Kiracı Oturuyor
Fiyat Durumu
Genel Fiyat
Yapı Durumu
İkinci El
Krediye Uygunluk
KREDIYE UYGUN
Yatırıma Uygunluk
Bilinmiyor
Takas
Yok
İstinye Boğaziçi sitesi B Bloklarında özel yapım 4 oda 1 salon 2 banyolu açık teraslı önü açık aydınlık doğalgaz kombili çatı dubleks daire.
 ( Daire 4. kat olup asansör yoktur, ancak merdivenler geniş ve çıkılması kolaydır. )
Daire kiracılıdır.
 Lütfen gelmeden önce randevu alınız.
Tek Yetkili:
Topaloğlu Emlak istinye ofisi
İç Özellikler
done
ADSL
done
H

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:199: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



2.000.000TL
fullscreen
İlanı Haritada Görüntüle
İlan Numarası
8495838
İlan Oluşturma Tarihi
17 Temmuz 2020
İlan Güncelleme Tarihi
06 Ağustos 2020
Türü
Konut
Kategorisi
Satılık
Yapı Tipi
Betonarme
Net Metrekare
600 M2
Brüt Metrekare
650 M2
Salon Metrekare
20 M2
Oda Sayısı
4+1
Binanın Yaşı
5-10
Bulunduğu Kat
Müstakil
Binanın Kat Sayısı
3
Isıtma Tipi
Kombi Doğalgaz
Aidat
0 TL
Kira Getirisi
0 TL
Banyo Sayısı
2
WC Sayısı
2
Site İçerisinde
Hayır
Eşya Durumu
Boş
Kullanım Durumu
Mal Sahibi Oturuyor
Fiyat Durumu
Genel Fiyat
Yapı Durumu
İkinci El
Krediye Uygunluk
KREDIYE UYGUN
Yatırıma Uygunluk
Yatırıma Uygun
Takas
Yok
TÜLAY EMLAK TAN'!!
SARIYER USKUMRUKÖY DE MERKEZDE RUHSATLI VE İMARLI 4 DAİRELİK
BAHÇELİ TOPLU TAŞIMAYA YOLA ÇOK YAKIN 600 M'2 BAHÇELİ 2 KATLI 
3 KAT RUHSATI VARDIR..
FAKAT 3 .KAT YAPTIRILAMADI
BAHÇEDE VERİMLİ BİR SU KUYUSU BULUNAN 
3 KATLI RUHSATI BULUNAN YAPIDIR..
ARSA İÇİNDEDE Kİ EV  KAT İRTİFAKLI OLUP KONUMU BAKIMDAN DEĞERLİDİR..
NOT:EKPERTİZİN BİÇTİĞİ RAKAM 2 MİLYON 550 BİN  

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:251: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


IndexError: single positional indexer is out-of-bounds

## Verinin Düzenlenmesi

#### Kaydettiğimiz csv dosyasını okuyoruz.

In [14]:
df_zingat = pd.read_csv("zingat3.csv")

In [15]:
df_zingat

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,Konut,Satılık,Betonarme,185 M2,220 M2,Belirtilmemiş,3+1,11-20,Bahçe Dublex,5,...,2,Evet,Boş,Mal Sahibi Oturuyor,Genel Fiyat,İkinci El,KREDIYE UYGUN,Yatırıma Uygun,Yok,1.585.000
1,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
2,Konut,Satılık,Betonarme,160 M2,185 M2,Belirtilmemiş,3+1,20 Ve Üzeri,4. Kat,6,...,Belirtilmemiş,Hayır,Belirtilmemiş,Mal Sahibi Oturuyor,Genel Fiyat,İkinci El,KREDIYE UYGUN,Bilinmiyor,Yok,6.500.000
3,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
4,Konut,Satılık,Betonarme,180 M2,260 M2,30 M2,6+2,20 Ve Üzeri,Müstakil,4,...,3,Hayır,Eşyalı,Mal Sahibi Oturuyor,Genel Fiyat,İkinci El,KREDIYE UYGUN,Belirtilmemiş,Yok,5.000.000
5,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
6,Konut,Satılık,Belirtilmemiş,130 M2,135 M2,Belirtilmemiş,3+1,20 Ve Üzeri,1. Kat,5,...,Belirtilmemiş,Hayır,Boş,Kiracı Oturuyor,Genel Fiyat,Belirtilmemiş,KREDIYE UYGUN,Bilinmiyor,Yok,500.000
7,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
8,Konut,Satılık,Betonarme,160 M2,170 M2,Belirtilmemiş,3+1,5-10,Düz Giriş,5,...,2,Hayır,Boş,Boş,Birim Fiyat,İkinci El,KREDIYE UYGUN,Yatırıma Uygun,Yok,1.300.000
9,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23


#### Görüldüğü üzere verisetinde gereksiz satırlar var. Bunlardan kurtuluyoruz

In [16]:
i = 1
while i <= 18:
    df_zingat.drop(i,inplace = True)
    i=i+2

#### Veri Setinde ki satır numaralarını düzenliyoruz ve ortaya çıkan gereksiz sütunu da siliyoruz.

In [18]:
df_zingat = df_zingat.reset_index()
df_zingat.drop("index", axis = 1, inplace = True)

#### Ve güzel bir şekilde veri setimizi hazırlamış olduk

In [19]:
df_zingat

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,Konut,Satılık,Betonarme,185 M2,220 M2,Belirtilmemiş,3+1,11-20,Bahçe Dublex,5,...,2,Evet,Boş,Mal Sahibi Oturuyor,Genel Fiyat,İkinci El,KREDIYE UYGUN,Yatırıma Uygun,Yok,1.585.000
1,Konut,Satılık,Betonarme,160 M2,185 M2,Belirtilmemiş,3+1,20 Ve Üzeri,4. Kat,6,...,Belirtilmemiş,Hayır,Belirtilmemiş,Mal Sahibi Oturuyor,Genel Fiyat,İkinci El,KREDIYE UYGUN,Bilinmiyor,Yok,6.500.000
2,Konut,Satılık,Betonarme,180 M2,260 M2,30 M2,6+2,20 Ve Üzeri,Müstakil,4,...,3,Hayır,Eşyalı,Mal Sahibi Oturuyor,Genel Fiyat,İkinci El,KREDIYE UYGUN,Belirtilmemiş,Yok,5.000.000
3,Konut,Satılık,Belirtilmemiş,130 M2,135 M2,Belirtilmemiş,3+1,20 Ve Üzeri,1. Kat,5,...,Belirtilmemiş,Hayır,Boş,Kiracı Oturuyor,Genel Fiyat,Belirtilmemiş,KREDIYE UYGUN,Bilinmiyor,Yok,500.000
4,Konut,Satılık,Betonarme,160 M2,170 M2,Belirtilmemiş,3+1,5-10,Düz Giriş,5,...,2,Hayır,Boş,Boş,Birim Fiyat,İkinci El,KREDIYE UYGUN,Yatırıma Uygun,Yok,1.300.000
5,Konut,Satılık,Belirtilmemiş,130 M2,150 M2,Belirtilmemiş,3+1,11-20,Düz Giriş,4,...,1,Hayır,Belirtilmemiş,Boş,Genel Fiyat,Belirtilmemiş,Krediye Uygun Değil,Belirtilmemiş,Bel,680.000
6,Konut,Satılık,Betonarme,135 M2,150 M2,25 M2,4+1,20 Ve Üzeri,4. Kat,5,...,2,Hayır,Boş,Kiracı Oturuyor,Genel Fiyat,İkinci El,KREDIYE UYGUN,Bilinmiyor,Yok,1.025.000
7,Konut,Satılık,Çelik,620 M2,650 M2,120 M2,6+3,0 (Yeni),Villa Tipi,4,...,5,Hayır,Belirtilmemiş,Boş,Genel Fiyat,Sıfır,KREDIYE UYGUN,Yatırıma Uygun,Yok,8.500.000
8,Konut,Satılık,Betonarme,600 M2,650 M2,20 M2,4+1,5-10,Müstakil,3,...,2,Hayır,Boş,Mal Sahibi Oturuyor,Genel Fiyat,İkinci El,KREDIYE UYGUN,Yatırıma Uygun,Yok,2.000.000
9,Konut,Satılık,Belirtilmemiş,128 M2,170 M2,Belirtilmemiş,3+1,5-10,10-20. Kat,30,...,2,Hayır,Boş,Kiracı Oturuyor,Birim Fiyat,İkinci El,KREDIYE UYGUN,Belirtilmemiş,Yok,2.250.000


#### Düzgün olan verimizi excel dosyası olarak kaydediyoruz

In [2]:
#df_zingat.to_excel("zingat2.xlsx")